In [ ]:
from dataclasses import asdict, dataclass
import random
from typing import List, Tuple

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from cyslf.optimize import optimize_player_assignment
from cyslf.models import Player, Team, League
from cyslf.scorers import *
from cyslf.utils import practice_fields, max_distance

%load_ext autoreload
%autoreload 2

In [ ]:
WEEKDAYS = ["M", "T", "W", "R", "F"]

In [ ]:
# SET UP LEAGUE WITH TOY DATA
random.seed(5)

# Find the mean location (for generating fake player data)
lats, longs = list(zip(*practice_fields))
mean_lat = np.mean(lats)
mean_long = np.mean(longs)
print(mean_lat, mean_long)

players = []
for i, letter in enumerate("ABCDEFGHIJKLMNOPQ"):
    grade = random.randint(4, 5)
    skill = random.randint(1, 5) if i != 0 else 10
    location = random.gauss(mean_lat, max_distance/5), random.gauss(mean_long, max_distance/5)
    players.append(Player(
        id=i,
        first_name=letter,
        last_name=letter,
        grade=grade,
        skill=skill,
        unavailable_days=random.choice(WEEKDAYS),
        location=location,
    ))

teams=[]
for i in range(1, 6):
    location = practice_fields[(i-1) % len(practice_fields)]
    teams.append(Team(name=str(i), practice_day=WEEKDAYS[i-1], location=location))    

# ASSIGN PlAYERS TO TEAMS    
league = League(teams=teams)
for i, player in enumerate(players[:-2]):
    # Optimized assignment
    optimize_player_assignment(player, league)
    print(
        f"~~~ {score_skill(league):.2f} "
        f"{score_grade(league):.2f} "
        f"{score_size(league):.2f} "
        f"{score_convenience(league):.2f} ~~~"
    )


# SHOW RESULTS
print(league)
for team in league.teams:
    if len(team.players) == 0:
        continue
    lats, longs = list(zip(*[p.location for p in team.players]))
    plt.scatter(lats, longs, s=25, label=team.name)
    
field_lats, field_longs = list(zip(*practice_fields))
plt.scatter(field_lats, field_longs, color="k", s=50)
plt.legend()

# Run on example data

In [ ]:
# I made this based on the example spreadsheets Jason gave me.
# Names didn't match, so I couldn't really join the tables.
data = pd.read_csv("input - combined.csv")
data.head()

In [ ]:
# Display some summary stats / plots
print("SKILL")
print(data.skill.value_counts())
print()

print("GRADE")
print(data.grade.value_counts())
print()

print("TEAMS")
print(data.team.value_counts())
print()

plt.scatter(data.longitude, data.latitude, marker=".")

sigma = 0.05
plt.xlim(data.longitude.median() - sigma, data.longitude.median() + sigma)
plt.ylim(data.latitude.median() - sigma, data.latitude.median() + sigma)

field_lats, field_longs = list(zip(*practice_fields))
plt.scatter(field_longs, field_lats, color="r", s=50)

In [ ]:
# SET UP LEAGUE
teams = []
for i, name in enumerate(data.team.unique()):
    if pd.isnull(name):
        continue
    location = practice_fields[i % len(practice_fields)]
    practice_day = WEEKDAYS[i % len(WEEKDAYS)]
    teams.append(Team(name=name, practice_day=practice_day, location=location))  
teams = sorted(teams, key=lambda t: t.name)
league = League(teams=teams)
print(league)
    
players = []
for i, row in data.iterrows():
    skill = row.skill
    if pd.isna(skill) or not skill[0].isdigit():
        continue
        # Ignore blank, Premier, Kick Start (ask Jason later)
    unavailable_days = []
    if not pd.isna(row.unavailable_days):
        for day in row.unavailable_days.split(","):
            clean_day = day.strip()
            if clean_day == "Thursday":
                unavailable_days.append("R")
            else:
                unavailable_days.append(clean_day[0])

    p = Player(
        id=i,
        first_name=row.first_name,
        last_name=row.last_name,
        grade=row.grade,
        skill=int(skill[0]),
        unavailable_days="".join(unavailable_days),
        location=(row.latitude, row.longitude)
    )
    players.append(p)
    
    # Add player to existing team
#     for team in teams:
#         if team.name == row.team:
#             team.add_player(p)
players = sorted(players, key=lambda p: -p.skill)
print(league)

In [ ]:
for i, player in enumerate(players):
    # Optimized assignment
    optimize_player_assignment(player, league)
    print(
        f"{i} / {len(players)} "
        f"~~~ {score_skill(league):.2f} "
        f"{score_grade(league):.2f} "
        f"{score_size(league):.2f} "
        f"{score_convenience(league):.2f} ~~~"
    )
#     print(league)

In [ ]:
league.to_csv("result.csv")

In [ ]:
# EXISTING TEAM STATS
# 18 Teams:
# Team Argentina   (F size=14, skill=2.71, grade=3.29)
# Team Belgium     (R size=15, skill=3.53, grade=3.67)
# Team Brazil      (F size=13, skill=3.23, grade=3.23)
# Team England     (W size=14, skill=3.57, grade=3.21)
# Team Eritrea     (W size=11, skill=3.00, grade=3.45)
# Team Ethiopia    (M size=15, skill=3.47, grade=3.20)
# Team Germany     (T size=13, skill=2.92, grade=3.38)
# Team Ireland     (M size=15, skill=3.13, grade=3.47)
# Team Italy       (M size=15, skill=3.60, grade=3.27)
# Team Ivory Coast (M size=12, skill=3.08, grade=3.75)
# Team Japan       (R size=14, skill=3.14, grade=3.57)
# Team Korea       (F size=11, skill=2.09, grade=3.64)
# Team Mexico      (W size=14, skill=3.21, grade=3.29)
# Team Netherlands (R size=15, skill=3.00, grade=3.33)
# Team Portugal    (T size=14, skill=3.50, grade=3.64)
# Team Spain       (T size=12, skill=3.33, grade=3.25)
# Team USA         (R size=13, skill=3.23, grade=3.38)
# Team Uruguay     (W size=14, skill=3.14, grade=3.36)